In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from torchvision import datasets
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
from utils.BP_neural_utils import bp_neural as bp

def save_parameters(file_name='parameters.npz', **args):
    _parameters = {}
    for key,value in args.items():
        _parameters[key] = value
    np.savez(file_name,**_parameters)
    print('Save file:',file_name)

In [ ]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, download=True)

train_imgs = train_dataset.data.numpy()
train_labels = train_dataset.targets.numpy()
test_imgs = test_dataset.data.numpy()
test_labels = test_dataset.targets.numpy()

batch = 5000
train_imgs = train_imgs[:batch]
train_labels = train_labels[:batch]

In [ ]:
a = bp(train_imgs,train_labels,n_hidden=100)
a.normalize_images(inplace=True)
a.one_hot(inplace=True)
a.init_params()
a.node_info()
error = []

In [ ]:
for i in tqdm(range(500)):
    a.front_prop()
    a.back_prop()
    a.update_params(lr=0.4)
    error.append(1-a.accuracy())
print(f'Train_error: {1-a.accuracy()}')
print(f'Test_error: {1-(a.predict(test_imgs.reshape(test_imgs.shape[0],-1).T) == test_labels).mean()}')
save_parameters('parameters.npz',w1=a.w1,w2=a.w2,b1=a.b1,b2=a.b2)
a.plot()

In [ ]:
plt.title(f'Current Error Rate:{1-a.accuracy()}')
plt.plot(error)

In [ ]:
index = np.random.randint(0,9999)
num = a.predict(test_imgs[index].reshape(1,-1).T)
plt.title(f'True:{test_labels[index]},Predicted:{num[0]}')
plt.imshow(test_imgs[index],cmap='gray')
plt.colorbar()
plt.show()

In [ ]:
parameters = np.load('parameters.npz')
a.w1 = parameters['w1']
a.b1 = parameters['b1']
a.w2 = parameters['w2']
a.b2 = parameters['b2']

In [ ]:
# opencv 手绘预测
import cv2

def draw(event,x,y):
    global ix,iy,drawing
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix,iy = x,y
    elif event ==cv2.EVENT_MOUSEMOVE:
        if drawing:
            cv2.circle(m_img,(x,y),25,(255,255,255),-1)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
    elif event == cv2.EVENT_RBUTTONDOWN:
        m_img[:,:] = 0

m_img = np.zeros((512,512,3),dtype=np.uint8)
cv2.namedWindow('image')
cv2.setMouseCallback('image',draw)
drawing = False

while True:
    gm_img = cv2.cvtColor(m_img,cv2.COLOR_BGR2GRAY)
    cv2.imshow('image',gm_img)
    res = cv2.resize(gm_img,(28,28),interpolation=cv2.INTER_AREA)
    # cv2.imshow('res',res)
    res = res.reshape(1,28*28)
    cv2.setWindowTitle('image','predicted num:' + str(a.predict(res.T)[0]))
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()